# EndSARS Twitter Analysis

Sentiment and Network Analysis of the #EndSARS protest movement that occured in 2020 in Nigeria. 

## SetUp

### Import Python Libraries

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import yaml
import os
import json
import re
import ast
import csv
import gensim
from searchtweets import ResultStream, gen_request_parameters, load_credentials, collect_results
# import fastText as ft
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime, timedelta


%matplotlib inline
pd.set_option("display.max_rows", 500)

### Connecting to Twitter

I suggest you use Twitter's official searchtweets api which is easy to use and reliable.
<br>In order to get past the tweets retrieval limit, you might also need to  use the premium search api.

- Create a hidden yaml file (.yaml) which contains details about your twitter api keys
- Create a variable that stores the twitter api credentials from the hiddem yaml file above.

**Keywords**: EndSARS, Soro Soke, Police Brutality, lekki toll gate, anti-robbery, lekki massacre, End bad governance,
End swat, buhari is a bad boy, sanwa-olu is a murderer, #LekkiMassacre

Helpful links:
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
- https://github.com/twitterdev/search-tweets-python/tree/v2#using-the-twitter-search-apis-python-wrapper

In [47]:
 search_args= load_credentials("tweetkeys1.yaml",
                                       yaml_key="search_tweets_v2",
                                       env_overwrite=False)


In [48]:
search_args  ['bearer_token']


'AAAAAAAAAAAAAAAAAAAAANE2NAEAAAAAS7YKh9sTQnSUY8ERjYLRiASf7Gw%3DOphmA1rPK5Xtc7BQcLAL9GRRas6mTFvEq7kd5U18ydBBsXj5M6'

In [49]:
querry1 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-01-01", end_time="2020-01-31",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss1 = collect_results(querry1,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss1[0:10]]
tweet_df= pd.DataFrame(tweetss1)
tweet_df.head()

@SpencerKarter #Disgraced #PrinceGeorgesCountyVA police officer #MichaelOwen has been charged w/ murder in the death of 43-year-old #WilliamGreen, who was shot 2 death while handcuffed in a police car. Reaction? #JusticeForWilliamGreen #EndPoliceBrutality #EndCorruptionNow

Pat John Paul Balneg engaged with the rice retailer at Buddymil Commercial in Brgy Laang,  Lagangilang, Abra regarding PRRDs directives on rice hoarders likewise reiterated anti-robbery and theft,  anti-trafficking in person. @abrappo @PNPCordillera https://t.co/5znWiQ5TJW

PSSg Christopher B Abella conducted Web Based Online Community Engagement Survey (WeBOCES) to the 8 clients of Editha Pancitan in Brgy Laang,  Lagangilang,  Abra. Likewise reminded them on anti-robbery and theft and road safety measure. 
@abrappo @PNPCordillera https://t.co/7w5oYv33Rc

#justiceforgoodness
#justiceforrapevictims
#endpolicebrutality
@a_nickabugu please tag relevant authorities.

Distribution of 30 IEC materials to market goers/ sel

,in_reply_to_user_id,entities,created_at,id,public_metrics,text,possibly_sensitive,author_id,lang,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",2020-01-30T23:09:11.000Z,1223020360523755520,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",@SpencerKarter #Disgraced #PrinceGeorgesCounty...,False,19043242,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"{'annotations': [{'start': 0, 'end': 19, 'prob...",2020-01-30T21:50:32.000Z,1223000568559370240,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Pat John Paul Balneg engaged with the rice ret...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"{'annotations': [{'start': 0, 'end': 24, 'prob...",2020-01-30T21:46:55.000Z,1222999657179013120,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",PSSg Christopher B Abella conducted Web Based ...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",2020-01-30T21:42:38.000Z,1222998578970406912,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",#justiceforgoodness\n#justiceforrapevictims\n#...,False,1190273440244228097,en,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"{'annotations': [{'start': 116, 'end': 122, 'p...",2020-01-30T21:42:16.000Z,1222998489270870016,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Distribution of 30 IEC materials to market goe...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
[print(tweet['text'], end='\n\n') for tweet in tweetss1[0:10]]
tweet_df= pd.DataFrame(tweetss1)
tweet_df.head()

@SpencerKarter #Disgraced #PrinceGeorgesCountyVA police officer #MichaelOwen has been charged w/ murder in the death of 43-year-old #WilliamGreen, who was shot 2 death while handcuffed in a police car. Reaction? #JusticeForWilliamGreen #EndPoliceBrutality #EndCorruptionNow

Pat John Paul Balneg engaged with the rice retailer at Buddymil Commercial in Brgy Laang,  Lagangilang, Abra regarding PRRDs directives on rice hoarders likewise reiterated anti-robbery and theft,  anti-trafficking in person. @abrappo @PNPCordillera https://t.co/5znWiQ5TJW

PSSg Christopher B Abella conducted Web Based Online Community Engagement Survey (WeBOCES) to the 8 clients of Editha Pancitan in Brgy Laang,  Lagangilang,  Abra. Likewise reminded them on anti-robbery and theft and road safety measure. 
@abrappo @PNPCordillera https://t.co/7w5oYv33Rc

#justiceforgoodness
#justiceforrapevictims
#endpolicebrutality
@a_nickabugu please tag relevant authorities.

Distribution of 30 IEC materials to market goers/ sel

,in_reply_to_user_id,entities,created_at,id,public_metrics,text,possibly_sensitive,author_id,lang,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",2020-01-30T23:09:11.000Z,1223020360523755520,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",@SpencerKarter #Disgraced #PrinceGeorgesCounty...,False,19043242,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"{'annotations': [{'start': 0, 'end': 19, 'prob...",2020-01-30T21:50:32.000Z,1223000568559370240,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Pat John Paul Balneg engaged with the rice ret...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"{'annotations': [{'start': 0, 'end': 24, 'prob...",2020-01-30T21:46:55.000Z,1222999657179013120,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",PSSg Christopher B Abella conducted Web Based ...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",2020-01-30T21:42:38.000Z,1222998578970406912,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",#justiceforgoodness\n#justiceforrapevictims\n#...,False,1190273440244228097,en,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"{'annotations': [{'start': 116, 'end': 122, 'p...",2020-01-30T21:42:16.000Z,1222998489270870016,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Distribution of 30 IEC materials to market goe...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
tweet_df.tail()

,in_reply_to_user_id,entities,created_at,id,public_metrics,text,possibly_sensitive,author_id,lang,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
3744,14879097,"{'hashtags': [{'start': 236, 'end': 244, 'tag'...",2020-01-01T01:04:24.000Z,1212177722350481409,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",@segalink Since they want our gallant policeme...,False,2371601656,en,"[{'type': 'replied_to', 'id': '121209789361786...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3745,NaN,"{'hashtags': [{'start': 0, 'end': 8, 'tag': 'E...",2020-01-01T00:54:02.000Z,1212175112067264512,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",#EndSARS \n#EndSARS \n\n#EndSARS \n#EndSARS \n...,False,71767071,en,"[{'type': 'quoted', 'id': '1212161578495287297'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3746,NaN,"{'hashtags': [{'start': 0, 'end': 8, 'tag': 'E...",2020-01-01T00:00:15.000Z,1212161578495287297,"{'retweet_count': 9, 'reply_count': 0, 'like_c...",#EndSARS \n#EndSARS \n\n#EndSARS \n#EndSARS \n...,False,427272900,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': '1930264094', 'name': 'Victor Olawuyi'...","[{'entities': {'urls': [{'start': 79, 'end': 1...","[{'full_name': 'Imo, Nigeria', 'country': 'Nig...",NaN,NaN,NaN,NaN
3748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1215266352459657217,1212161578495287297,288.0,NaN


In [52]:
querry2 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-02-01", end_time="2020-02-29",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss2 = collect_results(querry2,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss2[0:10]]
tweet_df1= pd.DataFrame(tweetss2)
tweet_df1.head()

With the way you’ve written your experience in this thread, it’s obvious you’ve always been hearing about #EndSARS but never cared because it never happened to you. 

Well, now you can care.... https://t.co/PJxbXhmc2p

Pray For 🇳🇬 #EndSars https://t.co/j5HVsIzWHI

@RuggedyBaba @PoliceNG I’d like to bring this to your notice and disapproving the suggestion of dismissal of #EndSARS bc it will only change nothing. I’ve encountered them but I had to obey each and every barbaric act the exhibit that day.

The @PoliceNG is not your friend😠. It's not just #EndSARS the whole police needs a total reform.

End this rubbish
#EndSARS 
#EndSarsNow 
#EndSarsNow 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS https://t.co/dZVRIwkgfH

The #EndSARS detention center has been operational for years, with so many horrible and horrific encounters and stories from  many Nigerians, for exposing then @EmekaUgwuonye was/is still being framed and detained just to keep him from talking about

,entities,author_id,lang,public_metrics,possibly_sensitive,text,id,created_at,geo,in_reply_to_user_id,referenced_tweets,users,places,tweets,newest_id,oldest_id,result_count,next_token
0,"{'urls': [{'start': 194, 'end': 217, 'url': 'h...",315543965,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,With the way you’ve written your experience in...,1233533291715792897,2020-02-28T23:23:49.000Z,{'place_id': '00817d75276a95cf'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'urls': [{'start': 21, 'end': 44, 'url': 'htt...",1101343279348699136,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,Pray For 🇳🇬 #EndSars https://t.co/j5HVsIzWHI,1233528009530806273,2020-02-28T23:02:49.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'hashtags': [{'start': 109, 'end': 117, 'tag'...",4199589491,en,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",False,@RuggedyBaba @PoliceNG I’d like to bring this ...,1233521597757247488,2020-02-28T22:37:21.000Z,NaN,61831375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'hashtags': [{'start': 49, 'end': 57, 'tag': ...",230540627,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,The @PoliceNG is not your friend😠. It's not ju...,1233516460166860801,2020-02-28T22:16:56.000Z,NaN,230540627,"[{'type': 'replied_to', 'id': '123351549196627...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'urls': [{'start': 122, 'end': 145, 'url': 'h...",1104833680680341504,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,End this rubbish\n#EndSARS \n#EndSarsNow \n#En...,1233513990485159936,2020-02-28T22:07:07.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
tweet_df1.tail()

,entities,author_id,lang,public_metrics,possibly_sensitive,text,id,created_at,geo,in_reply_to_user_id,referenced_tweets,users,places,tweets,newest_id,oldest_id,result_count,next_token
6021,"{'urls': [{'start': 118, 'end': 141, 'url': 'h...",1227208707530010624,en,"{'retweet_count': 7, 'reply_count': 0, 'like_c...",False,happening now : Emergency call. Fulani herdsme...,1231944780659273729,2020-02-24T14:11:38.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6022,"{'urls': [{'start': 118, 'end': 141, 'url': 'h...",1173946650764746753,en,"{'retweet_count': 14, 'reply_count': 1, 'like_...",False,Check out these top secrets that your Landlord...,1231944749491376128,2020-02-24T14:11:31.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6023,"{'urls': [{'start': 87, 'end': 110, 'url': 'ht...",3436779550,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,BREAKING: 3 Protesters Allegedly Killed By Pol...,1231944742675664896,2020-02-24T14:11:29.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'public_metrics': {'followers_count': 1357, ...","[{'name': 'Lagos', 'full_name': 'Lagos, Nigeri...","[{'created_at': '2020-02-24T13:44:20.000Z', 'l...",NaN,NaN,NaN,NaN
6025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1231954790923587589,1231942707330895874,478.0,b26v89c19zqg8o3fo74e0ln7g98vda1b1ni534y502mwt


In [54]:
querry3 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-03-01", end_time="2020-03-31",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss3 = collect_results(querry3,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss3[0:10]]
tweet_df3= pd.DataFrame(tweetss3)
tweet_df3.head()

@RuggedyBaba As in ehn......
#EndSARS go come arrest all of them.

SARS already patrolling everywhere. 
SARS is anti robbery but they will be brutally dealing with people that refuses to stay indoors.
Buhari has spoken so it is time we face the consequences for that.

@verified @jack @Twitter. This account @segalink has done enough to save lives in Africa and at a time like this we dont need his handle compromised, pls kindly verify to continue to save lives #check #PoliceBrutality #EndSARS #FixSociety #RP #COVID19Nigeria https://t.co/E8Du2PO5Dk

Imagine being arrested by sarz for nothing.. on the 30th of March, 10:59pm......after Buharis speech😂😂 #StayAtHomeAndStaySafe #StayHome #EndSARS #WhenCoronaVirusIsOver

People on Twitter fighting for folks out there steady 

From #EndSars, #WhereisBuhari and now unto #CutDataPrices
 
One guy was arguing like a  idiot last night, I told him to stfu. We brought that your Saintly president out from hiding. Dude knows only Instagram &amp; snapchat

,entities,author_id,lang,public_metrics,possibly_sensitive,in_reply_to_user_id,text,id,created_at,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,"{'hashtags': [{'start': 29, 'end': 37, 'tag': ...",956267665978544128,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,61831375,@RuggedyBaba As in ehn......\n#EndSARS go come...,1244637722624561153,2020-03-30T14:48:51.000Z,"[{'type': 'replied_to', 'id': '124463725951055...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'annotations': [{'start': 0, 'end': 3, 'proba...",344058319,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,NaN,SARS already patrolling everywhere. \nSARS is ...,1244590649082609665,2020-03-30T11:41:48.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'urls': [{'start': 259, 'end': 282, 'url': 'h...",1135448891883757574,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,63796828,@verified @jack @Twitter. This account @segali...,1244573580450566145,2020-03-30T10:33:59.000Z,"[{'type': 'quoted', 'id': '1244533219686481920'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'hashtags': [{'start': 103, 'end': 125, 'tag'...",837237194687414272,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,NaN,Imagine being arrested by sarz for nothing.. o...,1244565912910540801,2020-03-30T10:03:31.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'hashtags': [{'start': 61, 'end': 69, 'tag': ...",923262983735738369,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,NaN,People on Twitter fighting for folks out there...,1244548691941343233,2020-03-30T08:55:05.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
querry4 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-10-13", end_time="2020-10-21",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss4 = collect_results(querry4,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss4[0:10]]
tweet_df4= pd.DataFrame(tweetss4)
tweet_df4.head()

#UnitedNations #CNN #the commander who ordered the killings of of the youth doing there peaceful protest in Nigeria #GenocideAtLekkiTollGate #EndSARS https://t.co/PD0IerxKPC

@AishaYesufu As we all heart breaks, we remain gallant with our decision to set this country back on the right track. #ENDBADGOVERNANCE https://t.co/y1TmhCpChu

R.I.P to the brave youths who were on the frontline, we would never forget this! We would never forget you!! We love you!!! #AbolishSARS #EndSARS #BuhariResign #PowerToThePeople #NIGERIABLEEDS #SarsMustGoNow

#EndSARS #EndPoliceBrutalityinNigeraNOW #EndBadGoveranceInNigeria guys we can do better than this... just because we can't do much doesn't mean we can't spread awareness!! https://t.co/loCa7cvcKY

#EndSARS protesters in Nigeria just wanted govt to end police brutality, reform government &amp; make life bearable. The govt doubled down by slaughtering them. The Int’l community cannot be silent @UNHumanRights @SenateForeign @SenBooker @_AfricanUnion @Bar

,entities,public_metrics,possibly_sensitive,author_id,created_at,id,text,lang,referenced_tweets,in_reply_to_user_id,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,"{'urls': [{'start': 150, 'end': 173, 'url': 'h...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,1214969085060096001,2020-10-20T23:59:59.000Z,1318703543868641281,#UnitedNations #CNN #the commander who ordered...,en,"[{'type': 'quoted', 'id': '1318647975573393411'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'urls': [{'start': 136, 'end': 159, 'url': 'h...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",False,1210811673428119553,2020-10-20T23:59:59.000Z,1318703541993771010,"@AishaYesufu As we all heart breaks, we remain...",en,"[{'type': 'replied_to', 'id': '131867282504199...",2531005222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'hashtags': [{'start': 124, 'end': 136, 'tag'...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",False,877920219074875397,2020-10-20T23:59:59.000Z,1318703541939208192,R.I.P to the brave youths who were on the fron...,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'urls': [{'start': 172, 'end': 195, 'url': 'h...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",False,1006962454784552960,2020-10-20T23:59:59.000Z,1318703541909901319,#EndSARS #EndPoliceBrutalityinNigeraNOW #EndBa...,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'urls': [{'start': 284, 'end': 307, 'url': 'h...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,126971012,2020-10-20T23:59:59.000Z,1318703541519851521,#EndSARS protesters in Nigeria just wanted gov...,en,"[{'type': 'quoted', 'id': '1318684559807029249'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df = pd.concat([tweet_df, tweet_df1,tweet_df3,tweet_df4])
df.columns


Index(['in_reply_to_user_id', 'entities', 'created_at', 'id', 'public_metrics',
       'text', 'possibly_sensitive', 'author_id', 'lang', 'referenced_tweets',
       'geo', 'users', 'tweets', 'places', 'newest_id', 'oldest_id',
       'result_count', 'next_token'],
      dtype='object')

In [66]:
users= list(df['author_id'].dropna().unique()) 
users[0]

'19043242'

In [107]:
df.to_pickle("./tweets.pkl")



In [108]:
pd.read_pickle('tweets.pkl')

,in_reply_to_user_id,entities,created_at,id,public_metrics,text,possibly_sensitive,author_id,lang,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",2020-01-30T23:09:11.000Z,1223020360523755520,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",@SpencerKarter #Disgraced #PrinceGeorgesCounty...,False,19043242,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"{'annotations': [{'start': 0, 'end': 19, 'prob...",2020-01-30T21:50:32.000Z,1223000568559370240,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Pat John Paul Balneg engaged with the rice ret...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"{'annotations': [{'start': 0, 'end': 24, 'prob...",2020-01-30T21:46:55.000Z,1222999657179013120,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",PSSg Christopher B Abella conducted Web Based ...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",2020-01-30T21:42:38.000Z,1222998578970406912,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",#justiceforgoodness\n#justiceforrapevictims\n#...,False,1190273440244228097,en,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"{'annotations': [{'start': 116, 'end': 122, 'p...",2020-01-30T21:42:16.000Z,1222998489270870016,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Distribution of 30 IEC materials to market goe...,False,1639176810,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6021,NaN,"{'hashtags': [{'start': 45, 'end': 59, 'tag': ...",2020-10-20T23:50:49.000Z,1318701237613793280,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",Who can even sleep man... this shit is awful #...,False,47166530,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6022,1296121982849867776,"{'hashtags': [{'start': 60, 'end': 68, 'tag': ...",2020-10-20T23:50:49.000Z,1318701237173366786,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@ShalomAbel1 Please copy &amp; paste these han...,False,1244899110173454337,en,"[{'type': 'replied_to', 'id': '131869932587525...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6023,NaN,"{'urls': [{'start': 57, 'end': 80, 'url': 'htt...",2020-10-20T23:50:49.000Z,1318701236707872770,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Me and you both\n\nI am scared and angry and s...,False,1197822758429904900,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'verified': False, 'name': 'vikkinikki⭕', 'i...","[{'possibly_sensitive': False, 'public_metrics...","[{'full_name': 'Winnipeg, Manitoba', 'id': '08...",NaN,NaN,NaN,NaN


In [84]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def auth():
    return os.environ.get()




def create_url(users):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values
    user_ids = 'ids={},{},{}'.format(users[0],users[1],users[2])
    'ids='+','.join(users)
    user_fields = "user.fields=description,created_at,id,location,public_metrics,username,verified"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url ="https://api.twitter.com/2/users?{}&{}".format(user_ids, user_fields)
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = search_args ['bearer_token']
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


In [86]:
bearer_token = search_args['bearer_token']
url = create_url(users)
headers = create_headers(bearer_token)
json_response =connect_to_endpoint(url, headers)
print(json.dumps(json_response, indent=4, sort_keys=True))


200
{
    "data": [
        {
            "created_at": "2009-01-15T22:45:21.000Z",
            "description": "We were all heading for each other on a collision course, no matter what. Maybe some people are just meant to be in the same story. \u2764\ufe0f\ud83e\udde1\ud83d\udc9b\ud83d\udc9a\ud83d\udc99\ud83d\udc9c He/Him",
            "id": "19043242",
            "location": "Philadelphia",
            "name": "Tarrell Bellinger",
            "public_metrics": {
                "followers_count": 1018,
                "following_count": 5000,
                "listed_count": 56,
                "tweet_count": 87311
            },
            "username": "olderbrother21",
            "verified": false
        },
        {
            "created_at": "2013-08-02T00:37:07.000Z",
            "description": "",
            "id": "1639176810",
            "location": "Laang, Lagangilang, Abra",
            "name": "PNPAbraLagangilang",
            "public_metrics": {
                "follower

In [ ]:
import time
df = pd.DataFrame

In [87]:
json_response 

{'data': [{'description': 'We were all heading for each other on a collision course, no matter what. Maybe some people are just meant to be in the same story. ❤️🧡💛💚💙💜 He/Him',
   'id': '19043242',
   'name': 'Tarrell Bellinger',
   'location': 'Philadelphia',
   'created_at': '2009-01-15T22:45:21.000Z',
   'public_metrics': {'followers_count': 1018,
    'following_count': 5000,
    'tweet_count': 87311,
    'listed_count': 56},
   'verified': False,
   'username': 'olderbrother21'},
  {'description': '',
   'id': '1639176810',
   'name': 'PNPAbraLagangilang',
   'location': 'Laang, Lagangilang, Abra',
   'created_at': '2013-08-02T00:37:07.000Z',
   'public_metrics': {'followers_count': 879,
    'following_count': 246,
    'tweet_count': 12647,
    'listed_count': 3},
   'verified': False,
   'username': 'PNPLagangilang'},
  {'description': 'I am still your Voiceover Artist.    Everyone is not for me and that’s okay, except it’s business, of course.',
   'id': '1190273440244228097',
   

In [90]:
df1= pd.DataFrame(json_response['data'])
df1.tail(10)

,description,id,name,location,created_at,public_metrics,verified,username
0,We were all heading for each other on a collis...,19043242,Tarrell Bellinger,Philadelphia,2009-01-15T22:45:21.000Z,"{'followers_count': 1018, 'following_count': 5...",False,olderbrother21
1,,1639176810,PNPAbraLagangilang,"Laang, Lagangilang, Abra",2013-08-02T00:37:07.000Z,"{'followers_count': 879, 'following_count': 24...",False,PNPLagangilang
2,I am still your Voiceover Artist. Everyone ...,1190273440244228097,Lady_vee 👑,"Enugu, Nigeria",2019-11-01T14:24:57.000Z,"{'followers_count': 505, 'following_count': 37...",False,Lady_Vee_ver


In [94]:
querry6 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-01-01", end_time="2020-01-31",
                                user_fields = "description,created_at,id,location,public_metrics,username,verified",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss6 = collect_results(querry6,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss6[0:10]]
tweet_df6= pd.DataFrame(tweetss6)
tweet_df6.head()

@SpencerKarter #Disgraced #PrinceGeorgesCountyVA police officer #MichaelOwen has been charged w/ murder in the death of 43-year-old #WilliamGreen, who was shot 2 death while handcuffed in a police car. Reaction? #JusticeForWilliamGreen #EndPoliceBrutality #EndCorruptionNow

Pat John Paul Balneg engaged with the rice retailer at Buddymil Commercial in Brgy Laang,  Lagangilang, Abra regarding PRRDs directives on rice hoarders likewise reiterated anti-robbery and theft,  anti-trafficking in person. @abrappo @PNPCordillera https://t.co/5znWiQ5TJW

PSSg Christopher B Abella conducted Web Based Online Community Engagement Survey (WeBOCES) to the 8 clients of Editha Pancitan in Brgy Laang,  Lagangilang,  Abra. Likewise reminded them on anti-robbery and theft and road safety measure. 
@abrappo @PNPCordillera https://t.co/7w5oYv33Rc

#justiceforgoodness
#justiceforrapevictims
#endpolicebrutality
@a_nickabugu please tag relevant authorities.

Distribution of 30 IEC materials to market goers/ sel

,possibly_sensitive,public_metrics,created_at,id,text,lang,in_reply_to_user_id,entities,author_id,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,False,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",2020-01-30T23:09:11.000Z,1223020360523755520,@SpencerKarter #Disgraced #PrinceGeorgesCounty...,en,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",19043242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:50:32.000Z,1223000568559370240,Pat John Paul Balneg engaged with the rice ret...,en,NaN,"{'urls': [{'start': 250, 'end': 273, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:46:55.000Z,1222999657179013120,PSSg Christopher B Abella conducted Web Based ...,en,NaN,"{'urls': [{'start': 262, 'end': 285, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",2020-01-30T21:42:38.000Z,1222998578970406912,#justiceforgoodness\n#justiceforrapevictims\n#...,en,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",1190273440244228097,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:42:16.000Z,1222998489270870016,Distribution of 30 IEC materials to market goe...,en,NaN,"{'urls': [{'start': 266, 'end': 289, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
querry7= gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-02-01", end_time="2020-02-29",
                                user_fields = "description,created_at,id,location,public_metrics,username,verified",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss7 = collect_results(querry7,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss7[0:10]]
tweet_df7= pd.DataFrame(tweetss7)
tweet_df7.head()

With the way you’ve written your experience in this thread, it’s obvious you’ve always been hearing about #EndSARS but never cared because it never happened to you. 

Well, now you can care.... https://t.co/PJxbXhmc2p

Pray For 🇳🇬 #EndSars https://t.co/j5HVsIzWHI

@RuggedyBaba @PoliceNG I’d like to bring this to your notice and disapproving the suggestion of dismissal of #EndSARS bc it will only change nothing. I’ve encountered them but I had to obey each and every barbaric act the exhibit that day.

The @PoliceNG is not your friend😠. It's not just #EndSARS the whole police needs a total reform.

End this rubbish
#EndSARS 
#EndSarsNow 
#EndSarsNow 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS 
#EndSARS https://t.co/dZVRIwkgfH

The #EndSARS detention center has been operational for years, with so many horrible and horrific encounters and stories from  many Nigerians, for exposing then @EmekaUgwuonye was/is still being framed and detained just to keep him from talking about

,geo,author_id,possibly_sensitive,lang,entities,public_metrics,created_at,text,id,in_reply_to_user_id,referenced_tweets,users,places,tweets,newest_id,oldest_id,result_count,next_token
0,{'place_id': '00817d75276a95cf'},315543965,False,en,"{'hashtags': [{'start': 106, 'end': 114, 'tag'...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-02-28T23:23:49.000Z,With the way you’ve written your experience in...,1233533291715792897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1101343279348699136,False,en,"{'hashtags': [{'start': 12, 'end': 20, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-02-28T23:02:49.000Z,Pray For 🇳🇬 #EndSars https://t.co/j5HVsIzWHI,1233528009530806273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,4199589491,False,en,"{'hashtags': [{'start': 109, 'end': 117, 'tag'...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",2020-02-28T22:37:21.000Z,@RuggedyBaba @PoliceNG I’d like to bring this ...,1233521597757247488,61831375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,230540627,False,en,"{'hashtags': [{'start': 49, 'end': 57, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-02-28T22:16:56.000Z,The @PoliceNG is not your friend😠. It's not ju...,1233516460166860801,230540627,"[{'type': 'replied_to', 'id': '123351549196627...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1104833680680341504,False,en,"{'hashtags': [{'start': 17, 'end': 25, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-02-28T22:07:07.000Z,End this rubbish\n#EndSARS \n#EndSarsNow \n#En...,1233513990485159936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
querry8= gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-01-01", end_time="2020-03-31",
                                user_fields = "description,created_at,id,location,public_metrics,username,verified",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss8 = collect_results(querry8,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss8[0:10]]
tweet_df8= pd.DataFrame(tweetss8)
tweet_df8.head()

@RuggedyBaba As in ehn......
#EndSARS go come arrest all of them.

SARS already patrolling everywhere. 
SARS is anti robbery but they will be brutally dealing with people that refuses to stay indoors.
Buhari has spoken so it is time we face the consequences for that.

@verified @jack @Twitter. This account @segalink has done enough to save lives in Africa and at a time like this we dont need his handle compromised, pls kindly verify to continue to save lives #check #PoliceBrutality #EndSARS #FixSociety #RP #COVID19Nigeria https://t.co/E8Du2PO5Dk

Imagine being arrested by sarz for nothing.. on the 30th of March, 10:59pm......after Buharis speech😂😂 #StayAtHomeAndStaySafe #StayHome #EndSARS #WhenCoronaVirusIsOver

People on Twitter fighting for folks out there steady 

From #EndSars, #WhereisBuhari and now unto #CutDataPrices
 
One guy was arguing like a  idiot last night, I told him to stfu. We brought that your Saintly president out from hiding. Dude knows only Instagram &amp; snapchat

,public_metrics,in_reply_to_user_id,text,author_id,lang,created_at,entities,referenced_tweets,id,possibly_sensitive,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",61831375,@RuggedyBaba As in ehn......\n#EndSARS go come...,956267665978544128,en,2020-03-30T14:48:51.000Z,"{'mentions': [{'start': 0, 'end': 12, 'usernam...","[{'type': 'replied_to', 'id': '124463725951055...",1244637722624561153,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,SARS already patrolling everywhere. \nSARS is ...,344058319,en,2020-03-30T11:41:48.000Z,"{'annotations': [{'start': 0, 'end': 3, 'proba...",NaN,1244590649082609665,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",63796828,@verified @jack @Twitter. This account @segali...,1135448891883757574,en,2020-03-30T10:33:59.000Z,"{'annotations': [{'start': 82, 'end': 87, 'pro...","[{'type': 'quoted', 'id': '1244533219686481920'}]",1244573580450566145,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,Imagine being arrested by sarz for nothing.. o...,837237194687414272,en,2020-03-30T10:03:31.000Z,"{'hashtags': [{'start': 103, 'end': 125, 'tag'...",NaN,1244565912910540801,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,People on Twitter fighting for folks out there...,923262983735738369,en,2020-03-30T08:55:05.000Z,"{'annotations': [{'start': 10, 'end': 16, 'pro...",NaN,1244548691941343233,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
querry9= gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500,start_time ="2020-10-13", end_time="2020-10-21",
                                user_fields = "description,created_at,id,location,public_metrics,username,verified",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")

tweetss9 = collect_results(querry9,
                        max_tweets=6000,
                        result_stream_args=search_args )


[print(tweet['text'], end='\n\n') for tweet in tweetss9[0:10]]
tweet_df9= pd.DataFrame(tweetss9)
tweet_df9.head()

#UnitedNations #CNN #the commander who ordered the killings of of the youth doing there peaceful protest in Nigeria #GenocideAtLekkiTollGate #EndSARS https://t.co/PD0IerxKPC

@AishaYesufu As we all heart breaks, we remain gallant with our decision to set this country back on the right track. #ENDBADGOVERNANCE https://t.co/y1TmhCpChu

R.I.P to the brave youths who were on the frontline, we would never forget this! We would never forget you!! We love you!!! #AbolishSARS #EndSARS #BuhariResign #PowerToThePeople #NIGERIABLEEDS #SarsMustGoNow

#EndSARS #EndPoliceBrutalityinNigeraNOW #EndBadGoveranceInNigeria guys we can do better than this... just because we can't do much doesn't mean we can't spread awareness!! https://t.co/loCa7cvcKY

#EndSARS protesters in Nigeria just wanted govt to end police brutality, reform government &amp; make life bearable. The govt doubled down by slaughtering them. The Int’l community cannot be silent @UNHumanRights @SenateForeign @SenBooker @_AfricanUnion @Bar

,referenced_tweets,entities,possibly_sensitive,public_metrics,lang,id,author_id,created_at,text,in_reply_to_user_id,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,"[{'type': 'quoted', 'id': '1318647975573393411'}]","{'urls': [{'start': 150, 'end': 173, 'url': 'h...",False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",en,1318703543868641281,1214969085060096001,2020-10-20T23:59:59.000Z,#UnitedNations #CNN #the commander who ordered...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'type': 'replied_to', 'id': '131867282504199...","{'urls': [{'start': 136, 'end': 159, 'url': 'h...",False,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",en,1318703541993771010,1210811673428119553,2020-10-20T23:59:59.000Z,"@AishaYesufu As we all heart breaks, we remain...",2531005222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"{'hashtags': [{'start': 124, 'end': 136, 'tag'...",False,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",en,1318703541939208192,877920219074875397,2020-10-20T23:59:59.000Z,R.I.P to the brave youths who were on the fron...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"{'urls': [{'start': 172, 'end': 195, 'url': 'h...",False,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",en,1318703541909901319,1006962454784552960,2020-10-20T23:59:59.000Z,#EndSARS #EndPoliceBrutalityinNigeraNOW #EndBa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'type': 'quoted', 'id': '1318684559807029249'}]","{'urls': [{'start': 284, 'end': 307, 'url': 'h...",False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",en,1318703541519851521,126971012,2020-10-20T23:59:59.000Z,#EndSARS protesters in Nigeria just wanted gov...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df1 = pd.concat([tweet_df6, tweet_df7,tweet_df8,tweet_df9])
df1


,possibly_sensitive,public_metrics,created_at,id,text,lang,in_reply_to_user_id,entities,author_id,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,False,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",2020-01-30T23:09:11.000Z,1223020360523755520,@SpencerKarter #Disgraced #PrinceGeorgesCounty...,en,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",19043242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:50:32.000Z,1223000568559370240,Pat John Paul Balneg engaged with the rice ret...,en,NaN,"{'urls': [{'start': 250, 'end': 273, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:46:55.000Z,1222999657179013120,PSSg Christopher B Abella conducted Web Based ...,en,NaN,"{'urls': [{'start': 262, 'end': 285, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",2020-01-30T21:42:38.000Z,1222998578970406912,#justiceforgoodness\n#justiceforrapevictims\n#...,en,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",1190273440244228097,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:42:16.000Z,1222998489270870016,Distribution of 30 IEC materials to market goe...,en,NaN,"{'urls': [{'start': 266, 'end': 289, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6021,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236707872770,Me and you both\n\nI am scared and angry and s...,en,NaN,"{'urls': [{'start': 57, 'end': 80, 'url': 'htt...",1197822758429904900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6022,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236665896966,Do not forget that \n\n1. Curfew was ordered. ...,en,NaN,"{'urls': [{'start': 281, 'end': 304, 'url': 'h...",1063362715,"[{'type': 'quoted', 'id': '1318682299702104064'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6023,False,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236296757248,Pray for Nigeria 🇳🇬💔 #EndSARS #EndPoliceBrutal...,en,NaN,"{'urls': [{'start': 148, 'end': 171, 'url': 'h...",1298406162522673153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'public_metrics': {'followers_count': 8, 'fo...","[{'author_id': '157626908', 'lang': 'en', 'pub...","[{'country': 'Canada', 'geo': {'type': 'Featur...",NaN,NaN,NaN,NaN


In [104]:
df1.to_pickle("./user.pkl")


In [106]:
pd.read_pickle('user.pkl')

,possibly_sensitive,public_metrics,created_at,id,text,lang,in_reply_to_user_id,entities,author_id,referenced_tweets,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,False,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",2020-01-30T23:09:11.000Z,1223020360523755520,@SpencerKarter #Disgraced #PrinceGeorgesCounty...,en,26422223,"{'hashtags': [{'start': 15, 'end': 25, 'tag': ...",19043242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:50:32.000Z,1223000568559370240,Pat John Paul Balneg engaged with the rice ret...,en,NaN,"{'urls': [{'start': 250, 'end': 273, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:46:55.000Z,1222999657179013120,PSSg Christopher B Abella conducted Web Based ...,en,NaN,"{'urls': [{'start': 262, 'end': 285, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",2020-01-30T21:42:38.000Z,1222998578970406912,#justiceforgoodness\n#justiceforrapevictims\n#...,en,1190273440244228097,"{'hashtags': [{'start': 0, 'end': 19, 'tag': '...",1190273440244228097,"[{'type': 'replied_to', 'id': '122299799378110...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-30T21:42:16.000Z,1222998489270870016,Distribution of 30 IEC materials to market goe...,en,NaN,"{'urls': [{'start': 266, 'end': 289, 'url': 'h...",1639176810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6021,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236707872770,Me and you both\n\nI am scared and angry and s...,en,NaN,"{'urls': [{'start': 57, 'end': 80, 'url': 'htt...",1197822758429904900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6022,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236665896966,Do not forget that \n\n1. Curfew was ordered. ...,en,NaN,"{'urls': [{'start': 281, 'end': 304, 'url': 'h...",1063362715,"[{'type': 'quoted', 'id': '1318682299702104064'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6023,False,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2020-10-20T23:50:49.000Z,1318701236296757248,Pray for Nigeria 🇳🇬💔 #EndSARS #EndPoliceBrutal...,en,NaN,"{'urls': [{'start': 148, 'end': 171, 'url': 'h...",1298406162522673153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'public_metrics': {'followers_count': 8, 'fo...","[{'author_id': '157626908', 'lang': 'en', 'pub...","[{'country': 'Canada', 'geo': {'type': 'Featur...",NaN,NaN,NaN,NaN


In [ ]:
def get_dt_after(start_date, delta=timedelta(days=7)):
    """
    Create a function where you put in a date and it returns a date 7 days after in string format
    """

In [ ]:
def create_tweets_df():
    Create a function that Creates a dataframe from the tweets you've retrieved using the retieve_tweets function 
    you created above. This functions should also take a list of datest you want to get tweets from
    """


## Fetch and Clean Data

Use the functions you've created above to collect all the tweets

In [ ]:
dates_list = ["2020-12-01",  "2020-12-08"]

In [ ]:
tweets_df = create_tweets_df(dates_list, max_tweets=10000)
tweets_df.shape

Make sure to save the tweets after this process, use a file format that won't comprimise the characters in tweets

In [ ]:
def delete_duplicates(df):
    """
    Create a function that looks at the tweets in the tweets dataframe and other characters and delete 
    duplicates as sometimes we can get duplicate data/tweets. For example if I tweet something and someone else
    retweets it then this could possibly be a duplicate. Is this useful information? You need to decide of course.
    """

def preprocess_tweet(tweet):
    """
    Create a function that preprocess the text in a single tweet. Suggestions for this function are:
    - correct all multiple white spaces to a single white space
    - convert all urls to string "URL"
    - if the username isn't relevant then convert @username to "AT_USER"
    - converts emoji's to text description of the emoji
    """

In [ ]:
tweets_df = delete_duplicates(tweets_to_analyse)
tweets_df.reset_index(inplace=True, drop=True)
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess_tweet)

## Alt data

In [ ]:
df = pd.read_excel('https://query.data.world/s/rghc6hq5b32ertwgebgwuyx2lcpxvt')

## Tweets Analysis

Now it's time to analyse this data to try and get some interesting insights. Below are some questions you can answer to get started but the best Data Scientists are creative thinkers so think out of th ebox to decide what will be interesting insight

**What locations in Nigeria and around the world were the hashtags popular?**

**How far did the hashtag reach and how deep was this reach?**

**When the did movement start on social media? When did it reach critical mass?**

**What were the most popular words or terms used during the movement?**

**Who were the key influences and proponents involved in the movement?**

**What was the most popular tweet?**

**What is the general sentiment of the tweets about the movement? How has this changed from the start of the movement till now?**

**Were there any bots or bad actors spreading fake news around the movement?**

**What are some examples of this fake news?**

## Network Analysis

Using networkx or any other network graph library, create a social network to identifiy influential people of the movement

## Sentiment Modelling
Train model on labelled twitter sentiment data from another dataset
<br>Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to endsars tweets text

### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [ ]:
def prep_data(df, filename=""):
    """
    Create a functions that prepares the endsars data in a format that the fasttext model likes
    """

**Train Model**

**Apply Modelto Test data**

**Score Model**

**Check labels on example tweets**

### Apply trained fastText model to tweets df

In [ ]:
tweets_df['ft_predict'] = tweets_df.text.apply(ft_model.predict)
tweets_df["sentiment"] = tweets_df.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_df['sentiment_prob'] = tweets_df.ft_predict.apply(lambda x: x[1][0])

In [ ]:
def print_tweet_sentiment(index=0):
    """
    Create function where you either pass the index of the tweet in the tweets_df or a tweet and it returns the 
    sentiment for the tweet
    """